# 12. Docker Container 2

다룰 내용: Tensorflow 실행, volume

## 1. Tensorflow Container

* 지난 실험에서 `docker run -it --rm -p 8888:8888 tensorflow/tensorflow:latest-jupyter` 을 통해 Jupyter로 Tensorflow를 실행하는 과정에서 오류가 있었다.
* 오류가 발생하는 이유는 여러가지가 있겠지만 가능성 있는 오류로 몇 가지 작성하면 다음과 같다.
  1. 이미 해당 포트에 다른 컨테이너가 연결되어 있어 중복되는 경우
![port error](fig 12/fig12-1.png)
    * 해당 포트를 사용중인 컨테이너를 지운 뒤 실행하면 정상적으로 동작한다.
    * 컨테이너 제거는 `-rm` 혹은 Kitematic 을 통해 수행할 수 있다.
  2. 해당 포트를 Jupyter Notebook에서 사용 중인 경우
    * 별다른 오류는 뜨지 않지만 포트에 접속했을 때 컨테이너의 디렉토리가 아닌 Jupyter Notebook이 보여주는 로컬 디스크 내의 디렉토리만을 보여준다.
    * 해당 경우에는 Jupyter Notebook을 종료해야 한다. 아나콘다 프롬프트에서 `jupyter notebook list`를 입력하면 Jupyter Notebook이 사용중인 포트를 확인할 수 있다.
    * 지난 주에 발생한 오류는 여기에 해당했으며, Jupyter Notebook 을 실행하지 않고 PowerShell로 동일한 실험을 진행한 결과 8888 포트에서 Tensorflow가 잘 실행됨을 확인하였다.
![port error](fig 12/fig12-2.png)
* 포트는 일반적으로 통용되는 번호 또는 경우가 있으니 필요시 검색을 통해 확인할 수 있다.


## 2. volume

* 도커에서 데이터를 컨테이너 내부 대신 도커 엔진에 저장하고 싶을 때 volume의 공간을 사용할 수 있다.
* 호스트의 파일 시스템 내에 특정 영역을 도커가 관리하면서 사용한다. 리눅스에서는 해당 디렉토리에 접근이 가능하지만 다른 OS에서는 불가능하다.
* bind mount는 호스트가 사용하는 장치에 의존하는 반면, volume은 완전히 도커에 의해 관리받는다.
* 이러한 특징으로 volume이 bind mount보다 유리한 장점은 다음과 같다.
  1. 쉽게 백업하거나 옮길 수 있다.
  2. 도커 CLI나 API를 통해 관리할 수 있다.
  3. Linux와 Windows 컨테이너 모두에서 작동한다.
  4. 컨테이너를 복사하더라도 안전하게 데이터를 공유할 수 있다.
  5. 원격 호스트나 클라우드 제공자에게 데이터를 공유하여 암호화하거나 추가적인 기능을 부여할 수 있다.
  6. 컨테이너로부터 기존의 내용을 미리 채워올 수 있다.
![volume](fig 12/fig12-3.png)
* 읽기-쓰기 레이어인 컨테이너를 데이터의 저장소로 사용하는 것 보다 좋을 수 있다.
  1. 볼륨은 컨테이너의 용량을 키우지 않기 때문이다.
  2. 볼륨은 컨테이너가 사라지더라도 남아있기 때문이다.
  
* `-v` or `--mount` 를 적절하게 선택할 것
  * `-v` `--volume`은 독립적인 컨테이너에 사용된다.
  * `--mount`는 다중인 컨테이너에 사용된다. (독립적인 컨테이너도 가능)
  * 상세한 옵션을 설정하고자 할 때는 `--mount`, 단순한 설정은 `-v`
  
* `-v` `--volume`
  * 3개의 영역을 `:` 로 구분한다.
  * 첫 번째 영역은 volume의 이름으로 이름이 붙여진 경우는 이름을 입력하여야 하고, 익명인 경우는 생략한다. 이름은 호스트 장치에서는 중복되지 않는다.
  * 두 번째 영역은 컨테이너에 장착된 파일 또는 디렉토리의 경로(path)이다.
  * 세 번째 영역은 선택 사항으로, `-v`에서 사용할 옵션에 대한 내용을 입력한다. 옵션은 아래에서 추가로 다룰 예정이다.

* `--mount`
  * key-value 쌍으로 구성되어 있으며 `,`와 key-value 튜플로 구성되어 있다.
  * key의 순서는 중요하지 않으며 옵션이름이 어렵지 않다.
  * `type` 은 `bind` `volume` `tmpfs`가 있는데 이 장에서는 `volume`을 다룰 것이다.
  * `source` 또는 `src` volume의 이름과 같다. 이름이 있으면 입력하고, 없을 경우 생략한다.
  * `destination` `dst` `target`은 컨테이너에 장착될 파일 또는 디렉토리의 경로를 취급한다.
  * `readonly` 읽기전용 옵션이다.
  * `volume-opt` 에서 key-value 값을 여러 개 받을 수 있으며, option name을 key, 해당하는 option value를 value로 받는다.
  
* volume 생성하기 `docker volume create <volume name>`
![create volume](fig 12/fig12-4.png)
* volume 확인하기 `docker volume ls`
![list volume](fig 12/fig12-5.png)
* volume 조사하기 `docker volume inspect <volume name>`
![inspect volume](fig 12/fig12-6.png)
* volume 제거하기 `docker volume rm <volume name>`
![remove volume](fig 12/fig12-7.png)
* 컨테이너에 volume 장착하기
  * `myvol2`을 컨테이너의 `/app/`에 위치시킬 것
  * `docker run -d --name devtest --mount source=myvol2,target=/app nginx:latest` 
  ![myvol2](fig 12/fig12-8.png)
  * `--mount`의 경우 컨테이너 `devtest` 또는 volume `myvol2`이 없으면 구동되지 않는다.
  * `docker run -d --name devtest -v myvol2:/app nginx:latest`
  * `--mount`, `-v` 모두 같은 결과를 보인다.
  * `docker inspect devtest`로 확인할 수 있다.
  ![devtest](fig 12/fig12-9.png)
  * mount가 volume임을 확인할 수 있으며 원하는 source, destination을 가진다는 것과 mount가 read-write 상태임을 알 수 있다.
  * 컨테이너를 지우더라도 volume은 남아있으므로 원하지 않는다면 꼭 컨테이너와 함께 제거해주어야 한다.
  ![devtest](fig 12/fig12-10.png)
  
* 호스트 장치의 파일을 Docker 내에서 다루고 싶을 때에는 호스트 디렉토리를 장착하고, 컨테이너의 작업 디렉토리를 바꾸도록 한다.
  * `-v <hostDir>:<containerDir> -w <workDir>`

## 3. MYSQL

* 오픈소스 중 하나로 관계형 데이터베이스 관리 시스템이다.
* 웹 기반 애플리케이션을 위한 데이터베이스로 많이 사용된다.
* `docker run -d -p 3306:3306 -e MYSQL_ALLOW_EMPTY_PASSWORD=true --name mysql mysql:5.7` 
* 패스워드 없이 root계정을 만들기 위해 `-e MYSQL_ALLOW_EMPTY_PASSWORD` 설정
* 컨테이너 이름 `mysql`, 백그라운드 모드 `-d`
 ![run mysql](fig 12/fig12-11.png)
* MYSQL 테스트를 위해서 설치가 되어있어야 한다.

## 4. WordPress

* 블로그, 사이트, 전자상거래, 기업 비즈니스, 웹사이트 등으로 이용되는 플랫폼이다.
* PHP와 MySQL을 기반으로 한 컨텐츠 관리 시스템 오픈소스이다.
![wordpress](fig 12/fig12-12.png)
* MySQL 컨테이너에 WordPress 데이터베이스를 만들고 WordPress 컨테이너를 실행할 때 `--link`를 통해 MYSQL 컨테이너와 연결할 것이다.
* `docker run -d -p 8080:80 --link mysql:mysql -e WORDPRESS_DB_HOST=mysql -e WORDPRESS_DB_NAME=wp -e WORDPRESS_DB_USER=wp -e WORDPRESS_DB_PASSWORD=wp wordpress`
* `--link`는 환경변수 -e와 IP정보 -p를 자동으로 입력하므로 워드프레스 컨테이너가 MySQL 데이터베이스의 정보를 알 수 있게 된다.
* 호스트의 8080포트를 컨테이너의 80포트에 연결하고 여러 환경변수를 입력하여 설정한다.
![run wordpress](fig 12/fig12-13.png)
* MySQL 설치를 하지 않아 link 실험에 대한 구체적인 결과는 얻지 못했다.

다음에 할 것

* `--link` 테스트 할 다른 컨테이너 조사
* 도커 컨테이너 구축하기 (`Dockerfile` `requirements.txt` `app.py`)
https://docs.docker.com/get-started/part2/

* 도커 서버 구축
* https://miiingo.tistory.com/90
* https://www.slideshare.net/pyrasis/docker-fordummies-44424016
* https://ndb796.tistory.com/95
* https://noanswercode.tistory.com/1
* https://blog.hanumoka.net/2018/11/14/docker-20181114-docker-ubuntu-nginx/
* 보통 Ubuntu + nginx 웹서버를 구현하는 방법

참조

https://www.tensorflow.org/install/docker#download_a_tensorflow_docker_image

https://docs.docker.com/storage/volumes/

https://futurecreator.github.io/2018/11/16/docker-container-basics/

https://darkrasid.github.io/docker/container/volume/2017/05/10/docker-volumes.html

https://dololak.tistory.com/402?category=644234

https://jungwoon.github.io/docker/2019/01/13/Docker-3/

https://docs.docker.com/engine/reference/commandline/run/

https://ko.wikipedia.org/wiki/MySQL